In [1]:
import syft as sy

In [2]:
client = sy.login(email="info@openmined.org", password="changethis", port=8080)#, url='http://74.248.9.75')

ConnectionError: Failed to fetch http://localhost:8080/api/v2/metadata. Response returned with code 502

In [2]:
# third party
import numpy as np

mock_high = np.array([10, 11, 12, 13, 14])
private_high = np.random.rand(125_000_000 * 5)

In [3]:
from sys import getsizeof

In [4]:
getsizeof(private_high)

5000000112

In [5]:
from syft.serde.recursive import recursive_scheme

In [3]:
our_bytes = private_high.tobytes()

In [6]:
msg = recursive_scheme.new_message()
msg.init("fieldsName", 10)
msg.init("fieldsData", 10)

<capnp list builder [[], [], [], [], [], [], [], [], [], []]>

In [7]:

def chunk_bytes(
    field_obj, ser_func, field_name: str | int, builder
) -> None:
    data = ser_func(field_obj)
    CHUNK_SIZE = int(5.12e8)  # capnp max for a List(Data) field
    list_size = len(data) // CHUNK_SIZE + 1
    data_lst = builder.init(field_name, list_size)
    END_INDEX = CHUNK_SIZE
    for idx in range(list_size):
        START_INDEX = idx * CHUNK_SIZE
        END_INDEX = min(START_INDEX + CHUNK_SIZE, len(data))
        data_lst[idx] = data[START_INDEX:END_INDEX]



In [8]:
msg.fieldsName[0] = 'test'
chunk_bytes(private_high, lambda x: x.tobytes(), 0, msg.fieldsData)

In [5]:
private_high.nbytes / (10**9)

5.0

* system ~ 5gb
* object - 5gb
* bytes - 5gb

In [ ]:
a = private_high.tobytes()

In [6]:
action_object = sy.ActionObject.from_obj(private_high)

In [7]:
msg = sy.serialize(action_object)

TypeError: chunk_bytes() missing 1 required positional argument: 'builder'

In [ ]:
action_object.send(client)

Uploading progress: 100%|██████████| 1232/1232 [00:08<00:00, 152.17it/s]


```python
Pointer
```
array([0.76927523, 0.53988129, 0.80223722, ..., 0.05256553, 0.13019749,
       0.89295591])

In [ ]:
client.api.services.action.get(action_object.id)

ConnectionError: Failed to fetch metadata. Response returned with code 502

In [ ]:
assert False

AssertionError: 

In [ ]:
dataset_high = sy.Dataset(
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_high,
            data=private_high,
            shape=private_high.shape,
            mock_is_real=True,
        )
    ],
)

client.upload_dataset(dataset_high)

  0%|          | 0/1 [00:00<?, ?it/s]

Uploading: numpy-data


100%|██████████| 1/1 [00:33<00:00, 33.17s/it]


**WARNING**: The total size for asset: 'numpy-data' exceeds '512 MB'. This might result in failure to upload dataset. Please contact #support on OpenMined slack for further assistance.


ConnectionError: Failed to fetch metadata. Response returned with code 502

In [ ]:
client.api.node_name

'mynode'

In [ ]:
import syft as sy
from syft.store.blob_storage.seaweedfs import SeaweedFSClientConfig, SeaweedFSClient
from syft.types.blob_storage import CreateBlobStorageEntry
import np

config = SeaweedFSClientConfig(
        host="localhost",  # "http://localhost:8888"
        port=8333,
        access_key="admin",
        secret_key="admin",
        region="us-east-1",
        default_bucket_name="mynode",
        mount_port=4001,
    )
seaweed_client = SeaweedFSClient(config=config)
connection = seaweed_client.connect()

# from 0.1B to 0.5B of elements, it starts to be slow
# serialized data is from 0.8GB to 4GB
mock_high = np.random.random(200_000_000)
print(mock_high.nbytes / (10**9))
ao = sy.ActionObject.from_obj(mock_high)
data = ao.syft_action_data
storage_entry = CreateBlobStorageEntry.from_obj(data)
res = connection.allocate(obj=storage_entry)
print(res)

NameError: name 'np' is not defined